In [ ]:
%matplotlib notebook
from helpers.helpers import read_results_alignment

from helpers.geometry_helpers import make_avg_fragment_if_not_exists
from helpers.helpers import read_results_alignment
from helpers.plot_functions import plot_fragment_colored

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np
import pandas as pd
import math

In [ ]:
filenames = []
filenames.append(".\\results\\ArCI\\ArCI_R2CO_vdw.5\\ArCI_R2CO_vdw.5_aligned.csv")
filenames.append(".\\results\\ArCI\\ArCI_R2CO_vdw.5\\124_ArCI_R2CO_vdw.5_aligned.csv")
filenames.append(".\\results\\ArCI\\ArCI_R2CO_vdw.5\\426_ArCI_R2CO_vdw.5_aligned.csv")

dfs = []
for filename in filenames:
    dfs.append(read_results_alignment(filename))

In [ ]:
avg_fragments = []

for df in dfs:
    central_group_df = df[df.in_central_group]

    fragment = central_group_df.groupby('atom_label').agg({'atom_symbol': 'first', 'atom_x': 'mean',
                                                                  'atom_y': 'mean', 'atom_z': 'mean'}).reset_index()
    fragment_stds = central_group_df.groupby('atom_label').agg({'atom_x': 'std',
                                                                  'atom_y': 'std', 'atom_z': 'std'}).reset_index()
    fragment_stds.columns = [col + '_std' for col in fragment_stds.columns]

    fragment_total = pd.merge(fragment, fragment_stds, left_on='atom_label', right_on='atom_label_std')
    fragment_total = fragment_total.drop(columns="atom_label_std")
    
    avg_fragments.append(fragment_total)
    
    display(fragment_total)

In [ ]:
for i, avg_fragment in enumerate(avg_fragments):
    z = 2.58
    squared_datapoints = math.sqrt(len(dfs[i]))
    ci_z = z / squared_datapoints
    
    avg_fragment['atom_x_ci'] = avg_fragment['atom_x_std'] * ci_z / 2
    avg_fragment['atom_y_ci'] = avg_fragment['atom_y_std'] * ci_z / 2
    avg_fragment['atom_z_ci'] = avg_fragment['atom_z_std'] * ci_z / 2
    
    print(sum(avg_fragment.drop(columns=['atom_symbol', 'atom_x', 'atom_y', 'atom_z', 'atom_x_std', 'atom_y_std', 'atom_z_std']).mean()))
          
    print(avg_fragment.drop(columns=['atom_symbol', 'atom_x_std', 'atom_y_std', 'atom_z_std']).round(4).to_latex())
    print(avg_fragment.drop(columns=['atom_symbol', 'atom_x_std', 'atom_y_std', 'atom_z_std']).mean().round(4).to_latex())

display(avg_fragments[0])

In [ ]:
for avg_fragment in avg_fragments:
    # width of the bars
    bar_width = 0.25

    r1 = np.arange(len(fragment_total))
    r2 = [x + bar_width for x in r1]
    r3 = [x + bar_width for x in r2]

    plt.figure()
    plt.bar(r1, avg_fragment.atom_x, yerr=avg_fragment.atom_x_ci, width=bar_width,
            capsize=7, label='x', color='tab:blue')

    plt.bar(r2, avg_fragment.atom_y, yerr=avg_fragment.atom_y_ci, width=bar_width,
            capsize=7, label='y', color='tab:orange')

    plt.bar(r3, avg_fragment.atom_z, yerr=avg_fragment.atom_z_ci, width=bar_width,
            capsize=7, label='z', color='tab:green')

    plt.xticks([r + bar_width for r in range(len(avg_fragment))], [atom.atom_label for _, atom in avg_fragment.iterrows()])

    plt.xlabel('Atom coordinate')
    plt.ylabel('Coordinate value')

    plt.legend()
    plt.show()

In [ ]:
for avg_fragment in avg_fragments:
    plt.figure(figsize=(8, 4))
    plt.title("Confidence Intervals per sample mean")
    
    plt.bar(r1, avg_fragment.atom_x_std * ci_z / 2, width=bar_width,
            capsize=7, label='x', color='tab:blue')

    plt.bar(r2, avg_fragment.atom_y_std * ci_z / 2, width=bar_width,
            capsize=7, label='y', color='tab:orange')

    plt.bar(r3, avg_fragment.atom_z_std * ci_z / 2, width=bar_width,
            capsize=7, label='z', color='tab:green')


    bar = plt.bar(r1[-1] + 1, sum(avg_fragment.atom_x_ci)/len(avg_fragment), width=bar_width,
            capsize=7, color='tab:blue')
    
    bar = plt.bar(r2[-1] + 1, sum(avg_fragment.atom_y_ci)/len(avg_fragment), width=bar_width,
            capsize=7, color='tab:orange')

    bar = plt.bar(r3[-1] + 1, sum(avg_fragment.atom_z_ci)/len(avg_fragment), width=bar_width,
            capsize=7, color='tab:green')

    xtick_labels = [atom.atom_label for _, atom in avg_fragment.iterrows()]
    xtick_labels.append('Average')
    plt.xticks([r + bar_width for r in range(len(avg_fragment) + 1)], xtick_labels)

    plt.xlabel('Atom label')
    plt.ylabel('Coordinate value')

    plt.legend()
    plt.show()

In [ ]:
for avg_fragment in avg_fragments:

    fig = plt.figure()
    
    # plot the (average of the) central group
    plt.scatter(avg_fragment.atom_x, avg_fragment.atom_y)

    plt.xlabel('X axis')
    plt.ylabel('Y axis')

    plt.show()